In [1]:
import pyam
import pandas as pd

from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

Step 1: Read in the necessary data

In [3]:
df = pyam.IamDataFrame(
    Path(
        '../data/101_data.xlsx'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/101_data.xlsx
pyam.core - INFO: Reading meta indicators


<IPython.core.display.Javascript object>

Step 2: Filter out for the necessary variables that we will use.

In [4]:
df.variable

['AR6 Reanalysis|OSCARv3.2|Carbon Removal',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only',
 'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only']

<IPython.core.display.Javascript object>

In [5]:
df_vars = (
    df
    .filter(
        region='World',
        variable=[
            'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only',
            'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
            'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'
        ]
    )
)

<IPython.core.display.Javascript object>

In [6]:
df_vars.convert_unit(
    current='Gt CO2-equiv/yr',
    to='Gt CO2/yr',
    factor=1,
    inplace=True
)

<IPython.core.display.Javascript object>

Step 3: Aggregate the gross emissions

In [7]:
df_vars.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Emissions|Gross',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
        'AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only'
    ],
    append=True
)

<IPython.core.display.Javascript object>

Step 4: Split the dataframe into two parts:
* The net GHG emissions that we will use to estimate the total mitigation effort
* The two component mitigation options we will use for proportions.

In [8]:
df_total_mitigation = (
    df_vars
    .filter(variable='AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only')
    .timeseries()
)

<IPython.core.display.Javascript object>

In [9]:
df_components_mitigation = (
    df_vars
    .filter(
        variable=[
            'AR6 Reanalysis|OSCARv3.2|Emissions|Gross',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'
        ]
    )
    .offset(year=2020)
    .timeseries()
)

<IPython.core.display.Javascript object>

Step 5: Pull out the net zero CO2 years

In [10]:
netzero_years = df_vars.meta.loc[:, 'Year of netzero CO2 emissions (Harm-Infilled) table']

<IPython.core.display.Javascript object>

In [11]:
netzero_years.fillna(2100, inplace=True)

<IPython.core.display.Javascript object>

Step 6: First, let us get the 2020 and net zero CO2 values of the net GHG emissions.

In [12]:
net_ghg_2020 = (
    pd.DataFrame(df_total_mitigation[2020])
    .droplevel(['region', 'variable', 'unit'])
    .round(0)
)

<IPython.core.display.Javascript object>

In [13]:
net_ghg_netzero = (
    pd.DataFrame(
        df_total_mitigation.apply(
            lambda x: x[netzero_years.loc[x.name[0:2]]],
            axis=1
        )
    )
    .droplevel(['region', 'variable', 'unit'])
    .round(0)
    .rename(
        columns={0:'netzero'}
    )
)

<IPython.core.display.Javascript object>

In [14]:
net_ghg_compiled = pd.concat([net_ghg_2020, net_ghg_netzero], axis=1)

<IPython.core.display.Javascript object>

In [15]:
net_ghg_compiled.loc[:,'Category'] = df_vars.meta.loc[:, 'Category']

<IPython.core.display.Javascript object>

In [16]:
net_ghg_compiled

2020  netzero Category
model             scenario                                       
AIM/CGE 2.0       SSP1-26                  51.0      3.0       C3
                  SSP4-26                  58.0      7.0       C3
AIM/CGE 2.1       CD-LINKS_NPi2020_1000    53.0      6.0       C3
                  CD-LINKS_NPi2020_400     53.0     -3.0       C1
                  CO_Bridge                55.0      7.0       C3
...                                         ...      ...      ...
WITCH-GLOBIOM 4.4 CD-LINKS_INDC2030i_1600  54.0      5.0       C3
                  CD-LINKS_NDC2030i_1000   54.0      4.0       C2
                  CD-LINKS_NPi2020_1000    56.0      4.0       C1
                  CD-LINKS_NPi2020_1600    56.0      5.0       C3
                  CD-LINKS_NPi2020_400     56.0      4.0       C1

[401 rows x 3 columns]

<IPython.core.display.Javascript object>

Step 7: Compile a dataframe with the difference between 2020 and net zero for the two mitigation component variables.

In [17]:
components_compiled = (
    df_components_mitigation.apply(
        lambda x: x[netzero_years.loc[x.name[0:2]]],
        axis=1
    )
    .droplevel(['region', 'unit'])
    .unstack(level=-1)
    .rename(
        columns={
            'AR6 Reanalysis|OSCARv3.2|Emissions|Gross':'reduce',
            'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total':'remove'
        }
    )
)

<IPython.core.display.Javascript object>

In [18]:
components_compiled.loc[:, 'remove'] *= -1

<IPython.core.display.Javascript object>

In [19]:
components_compiled

variable                                      remove     reduce
model             scenario                                     
AIM/CGE 2.0       SSP1-26                  -7.675157 -39.924308
                  SSP4-26                  -7.395017 -43.153110
AIM/CGE 2.1       CD-LINKS_NPi2020_1000   -12.240858 -35.431404
                  CD-LINKS_NPi2020_400    -16.143932 -40.820973
                  CO_Bridge                -7.373163 -40.490306
...                                              ...        ...
WITCH-GLOBIOM 4.4 CD-LINKS_INDC2030i_1600 -12.900026 -36.657457
                  CD-LINKS_NDC2030i_1000   -9.897303 -40.806546
                  CD-LINKS_NPi2020_1000    -9.834387 -42.161808
                  CD-LINKS_NPi2020_1600   -10.857048 -39.866954
                  CD-LINKS_NPi2020_400     -8.520957 -44.066332

[401 rows x 2 columns]

<IPython.core.display.Javascript object>

Step 8: Create a composite dataframe that we will use to plot stuff. Write it out for plotting.

In [20]:
figure2_panelc = pd.concat(
    [net_ghg_compiled, components_compiled],
    axis=1
)

<IPython.core.display.Javascript object>

In [21]:
figure2_panelc.to_csv(
    Path(
        '../data/104_figure2_panelc.csv'
    )
)

<IPython.core.display.Javascript object>

Step 9: Calculate the proportion of net GHG reductions met via reductions.

In [22]:
figure2_panelc.loc[:, 'total_reduction'] = (
    figure2_panelc.loc[:, 'netzero']
    -
    figure2_panelc.loc[:, 2020]
)

<IPython.core.display.Javascript object>

In [23]:
figure2_panelc

2020  netzero Category     remove  \
model             scenario                                                     
AIM/CGE 2.0       SSP1-26                  51.0      3.0       C3  -7.675157   
                  SSP4-26                  58.0      7.0       C3  -7.395017   
AIM/CGE 2.1       CD-LINKS_NPi2020_1000    53.0      6.0       C3 -12.240858   
                  CD-LINKS_NPi2020_400     53.0     -3.0       C1 -16.143932   
                  CO_Bridge                55.0      7.0       C3  -7.373163   
...                                         ...      ...      ...        ...   
WITCH-GLOBIOM 4.4 CD-LINKS_INDC2030i_1600  54.0      5.0       C3 -12.900026   
                  CD-LINKS_NDC2030i_1000   54.0      4.0       C2  -9.897303   
                  CD-LINKS_NPi2020_1000    56.0      4.0       C1  -9.834387   
                  CD-LINKS_NPi2020_1600    56.0      5.0       C3 -10.857048   
                  CD-LINKS_NPi2020_400     56.0      4.0       C1  -8.520957   

                                              reduce  total_reduction  
model             scenario                                             
AIM/CGE 2.0       SSP1-26                 -39.924308            -48.0  
                  SSP4-26                 -43.153110            -51.0  
AIM/CGE 2.1       CD-LINKS_NPi2020_1000   -35.431404            -47.0  
                  CD-LINKS_NPi2020_400    -40.820973            -56.0  
                  CO_Bridge               -40.490306            -48.0  
...                                              ...              ...  
WITCH-GLOBIOM 4.4 CD-LINKS_INDC2030i_1600 -36.657457            -49.0  
                  CD-LINKS_NDC2030i_1000  -40.806546            -50.0  
                  CD-LINKS_NPi2020_1000   -42.161808            -52.0  
                  CD-LINKS_NPi2020_1600   -39.866954            -51.0  
                  CD-LINKS_NPi2020_400    -44.066332            -52.0  

[401 rows x 6 columns]

<IPython.core.display.Javascript object>

In [24]:
figure2_panelc.loc[:, 'prop_reduce'] = (
    figure2_panelc.loc[:, 'reduce'] * 100
    /
    figure2_panelc.loc[:, 'total_reduction']
).round(0)

<IPython.core.display.Javascript object>

In [25]:
summary_stats = (
    figure2_panelc
    .groupby('Category')['prop_reduce']
    .describe()
)

<IPython.core.display.Javascript object>

Step 10: This backs up the sentence we use in the assessment:

However, across all the categories of pathways, over 80% of the net GHG reductions between 2020 and global net zero CO2 are achieved by cuts in gross CO2 and non-CO2 emissions (Figure 1c).

In [26]:
summary_stats

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
C1,70.0,84.142857,6.649149,72.0,79.25,85.0,88.0,102.0
C2,102.0,81.852941,7.035127,53.0,77.00,83.0,87.0,93.0
C3,229.0,81.502183,7.337494,52.0,75.00,83.0,87.0,101.0


<IPython.core.display.Javascript object>